In [2]:
import os
import json
import h5py
import numpy as np

import robomimic
import robomimic.utils.file_utils as FileUtils

# the dataset registry can be found at robomimic/__init__.py
from robomimic import DATASET_REGISTRY
import os
# print(os.environ['LD_LIBRARY_PATH'])
# print(list(os.environ))
os.environ['LD_LIBRARY_PATH'] = '/usr/local/cuda-12.1/lib64:/home/omniverse/.mujoco/mujoco210/bin:/usr/lib/nvidia:/home/omniverse/workspace/PyRep/CoppeliaSim_Edu_V4_1_0_Ubuntu20_04'
%cd /home/omniverse/workspace/robomimic/

/home/omniverse/workspace/robomimic


In [3]:
dataset_path = "./tmp/output.hdf5"
dataset_path = "./datasets/can/random_brown/low_dim_sparse_v141.hdf5"
agent_path = "cluster_output/mrl_trained_models_ds/mrl_can_ph/20240913173309/models/model_epoch_2000.pth"

# open file
f = h5py.File(dataset_path, "r")
# each demonstration is a group under "data"
demos = list(f["data"].keys())
num_demos = len(demos)

print("hdf5 file {} has {} demonstrations".format(dataset_path, num_demos))

hdf5 file ./datasets/can/random_brown/low_dim_sparse_v141.hdf5 has 10 demonstrations


# Visualizing demonstration trajectories

In [4]:
import robomimic.utils.env_utils as EnvUtils

env_meta = json.loads(f["data"].attrs["env_args"])
# create simulation environment from environment metedata
env = EnvUtils.create_env_from_metadata(
    env_meta=env_meta, 
    render=False,            # no on-screen rendering
    render_offscreen=True,   # off-screen rendering to support rendering video frames
    env_name= None,
    use_image_obs= False,
    use_depth_obs= False,
    goal_path= None
)




[robosuite WARNING] No private macro file found! (macros.py:53)
[robosuite WARNING] It is recommended to use a private macro file (macros.py:54)


[robosuite WARNING] To setup, run: python /home/omniverse/workspace/robosuite/robosuite/scripts/setup_macros.py (macros.py:55)


Created environment with name PickPlaceCan
Action size is 7


In [9]:
import robomimic.utils.obs_utils as ObsUtils
import imageio


# We normally need to make sure robomimic knows which observations are images (for the
# data processing pipeline). This is usually inferred from your training config, but
# since we are just playing back demonstrations, we just need to initialize robomimic
# with a dummy spec.
dummy_spec = dict(
    obs=dict(
            low_dim=["robot0_eef_pos"],
            rgb=[],
        ),
)
ObsUtils.initialize_obs_utils_with_obs_specs(obs_modality_specs=dummy_spec)

download_folder = "."

# prepare to write playback trajectories to video
video_path = os.path.join(download_folder, "playback.mp4")
video_writer = imageio.get_writer(video_path, fps=20)

def playback_trajectory(demo_key):
    """
    Simple helper function to playback the trajectory stored under the hdf5 group @demo_key and
    write frames rendered from the simulation to the active @video_writer.
    """
    
    # robosuite datasets store the ground-truth simulator states under the "states" key.
    # We will use the first one, alone with the model xml, to reset the environment to
    # the initial configuration before playing back actions.
    init_state = f["data/{}/states".format(demo_key)][0]
    model_xml = f["data/{}".format(demo_key)].attrs["model_file"]
    initial_state_dict = dict(states=init_state, model=model_xml)
    
    # reset to initial state
    env.reset_to(initial_state_dict)
    
    # playback actions one by one, and render frames
    actions = f["data/{}/actions".format(demo_key)][:]
    for t in range(actions.shape[0]):
        env.step(actions[t])
        video_img = env.render(mode="rgb_array", height=512, width=512, camera_name="agentview")
        video_writer.append_data(video_img)


============= Initialized Observation Utils with Obs Spec =============

using obs modality: low_dim with keys: ['robot0_eef_pos']
using obs modality: rgb with keys: []


In [10]:
for ep in demos[:]:
    print("Playing back demo key: {}".format(ep))
    playback_trajectory(ep)

# done writing video
video_writer.close()

Playing back demo key: demo_0
Playing back demo key: demo_1
Playing back demo key: demo_2
Playing back demo key: demo_3
Playing back demo key: demo_4
Playing back demo key: demo_5
Playing back demo key: demo_6
Playing back demo key: demo_7
Playing back demo key: demo_8
Playing back demo key: demo_9


In [11]:
from IPython.display import Video
Video(video_path, embed=True)